In [1]:
import ranking
import salvar_resultados as sr
import funcoes_st as fst

import pandas as pd
import numpy as np
import string

import time

In [2]:
#import torch
#torch.cuda.empty_cache()

In [3]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
#arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_val = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_val.append(df_val)
    lista_df_teste.append(df_teste)

# Usando os modelos ST

In [4]:
flag_st = True
if flag_st == True:

    nome_modelo, apelido_modelo = fst.setar_modelo(0)
    tam = 51

    dir_metricas = f"Dados/Resultados/Ranqueado/ST-{apelido_modelo}"

    for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

        '''df_teste = df_teste[:tam]
        df_treino = df_teste[:tam]
        df_val = df_teste[:tam]'''

        df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')
        
        inicio_tempo = time.time()
        embedding = fst.pipeline_st(df_treino, df_val, df_concat, nome_modelo)
        resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)

        final_tempo = time.time()

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

Epoch: 100%|██████████| 1/1 [04:45<00:00, 285.34s/it]
c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Resultados

In [ ]:
    a

In [5]:
arquivos = ['ale_1_1']#, 'ale_5_1', 'hn_1_1', 'hn_5_1']
metodo = "ST-dbm-v1"

lista_df_resultado = []
for arquivo in arquivos:

    df_r = pd.read_csv(f"Dados/Resultados/Ranqueado/{metodo}/{arquivo}_métricas.csv")
    lista_df_resultado.append(df_r)

In [6]:
df_r[["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,548.000000,548.000000,548.000000,548.000000,548.000000
mean,0.755474,0.945255,0.996350,2.830292,0.820185
std,0.430198,0.227689,0.060357,6.827627,0.327066
min,0.000000,0.000000,0.000000,1.000000,0.011765
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,85.000000,1.000000
